In [1]:
import pandas as pd
import numpy as np
from numba import jit, float32, float64

In [2]:
def mdd(ccr):
    r = np.expand_dims(np.exp(ccr.cumsum()), axis=1)
    return - np.tril(r.dot(1 / r.T) - 1).min()

In [3]:
@jit(float32(float32[:]), cache=True)
def numba_mdd(ccr):
    r = np.expand_dims(np.exp(ccr.cumsum()), axis=1)
    return - np.tril(r.dot(1 / r.T) - 1).min()

In [4]:
def max_dd_loop(returns):
    max_so_far = None
    r = np.exp(returns.cumsum())
    for idx, r_start in enumerate(r):
        for r_end in r[idx:]:
            current = r_end/r_start - 1
            if (max_so_far is None) or (current < max_so_far):
                max_so_far = current
    return max_so_far

In [5]:
@jit(float32(float64[:]), cache=True)
def numa_max_dd_loop(returns):
    max_so_far = None
    r = np.exp(returns.cumsum())
    for idx, r_start in enumerate(r):
        for r_end in r[idx:]:
            current = r_end/r_start - 1
            if (max_so_far is None) or (current < max_so_far):
                max_so_far = current
    return -max_so_far

In [54]:
np.random.seed(123)
hdf_file = 'D:/Algorithmic Trading/NEATrader/TrainingDataHDF/ohlcv_data.h5'
df = pd.read_hdf(hdf_file, 'minute/AAPL')
ccr = np.log(df.ix[:10000, 'Close']).diff().dropna().values
#returns.dtype = 'float32'
print type(ccr), ccr.shape

<type 'numpy.ndarray'> (9999L,)


In [ ]:
#print mdd(ccr), max_dd_loop(ccr)

In [ ]:
%timeit mdd(ccr)

In [ ]:
#%timeit numba_mdd(ccr)

In [ ]:
#%timeit max_dd_loop(ccr)

In [ ]:
%timeit numa_max_dd_loop(ccr)